In [63]:
import matplotlib.pyplot as plt
import matplotlib
import seaborn
from pylab import *
%matplotlib inline 

In [1]:
import csv
import json
from pymongo import MongoClient
import pandas as pd
import glob, os
from tqdm import tqdm
import pickle

def find_map(demo_info):
    for line in demo_info:
        if line.find(".bsp") != -1:
            match_map = line[line.find('maps/')+5:line.find('.bsp')]
            break
    return match_map


def player_info(demo_info):
    players = []
    for line in range(len(demo_info)):
        if demo_info[line].find("adding:player info:") != -1:
            player_dict = {}
            i = 1
            while demo_info[line+i] != "adding:player info:": # or demo_info[line+i].find(":") != -1
                if demo_info[line+i].find(":") == -1:
                    break
                player_dict[demo_info[line+i].split(':')[0].strip()] = demo_info[line+i].split(':')[1].strip()
                i += 1
            players.append(player_dict)
    return players


def event_org(event_string):
    '''
    Organize events by type
    '''
    sub_dict = {}
    if event_string.split(':')[0].strip() == 'position':
        sub_dict['x'] = event_string.split(':')[1].split(',')[0]
        sub_dict['y'] = event_string.split(':')[1].split(',')[1]
        sub_dict['z'] = event_string.split(':')[1].split(',')[2]
    elif event_string.split(':')[0].strip() == 'facing':
        sub_dict['pitch'] = event_string.split(':')[2].split(',')[0].strip()
        sub_dict['yaw'] =  event_string.split(':')[3].split(',')[0].strip()
    else:
        sub_dict[event_string.strip().split(':')[0].strip()] = event_string[len(event_string.split(':')[0])+2:].strip()
    return sub_dict


def parent_child(text_list):
    '''
    Determines how deep the item is in the dictionary
    For C++ class structure
    '''
    depth = 0
    sub_list = []
    event_list = []
    for line in text_list:
        previous_depth = depth
        if line == '{':
            depth += 1
        elif line == '}':
            depth -= 1
            if depth == 0:
                event_list.append(event_dict)
        elif depth == 0:
            event_dict = {'event_name': line}
        elif depth > 0:
            try: 
                event_dict.update(event_org(line))
            except:
                print line
                print type(line)
                print event_dict
                break
    return event_list

def events_to_dict(players, events, match_name, match_number, map_name):
    event_list = []
    round_number = 1
    event_number = 0
    round_prestart = 0
    round_end = 0
    for event in events:
        event_number += 1
        event['match'] = match_name
        event['match_number'] = match_number
        event['map_name'] = map_name
        if event['event_name'] in relevant_events:
            event['event_number'] = event_number
            if event['event_name'] == 'round_end':
                round_end = event_number
            if 'userid' in event.keys():
                if event['userid'] in [x['name'] + ' ' + '(id:' + x['userID'] + ')' for x in players]:
                    event['steamid'] = [x['xuid'] for x in players if x['name'] + ' ' + '(id:' + x['userID'] + ')' == event['userid']][0]
                #event['userid'] = event['userid'].split(' (id:')[0]
            if 'attacker' in event.keys():
                if event['attacker'] in [x['name'] + ' ' + '(id:' + x['userID'] + ')' for x in players]:
                    event['attacker_steamid'] = [x['xuid'] for x in players if x['name'] + ' ' + '(id:' + x['userID'] + ')' == event['attacker']][0]
                #event['attacker'] = event['attacker'].split(' (id:')[0]
            if event['event_name'] == 'player_spawn':
                if 'teamnum' in event.keys():
                    if event['teamnum'] == '2':
                        event['team'] = 'T'
                    elif event['teamnum'] == '3':
                        event['team'] = 'CT'
                    elif event['teamnum'] not in ['2', '3', 2, 3]:
                        continue
                if 'team' in event.keys():
                    if event['team'] == '2':
                        event['team'] = 'T'
                    elif event['team'] == '3':
                        event['team'] = 'CT'
                    elif event['team'] not in ['T', 'CT']:
                        continue
            event['round_number'] = round_number
            if event['event_name'] == 'round_prestart' and round_prestart < round_end:
                round_number += 1
            if event['event_name'] == 'round_prestart':
                round_prestart = event_number
            event_list.append(event)
    return event_list


def find_match_begin(events):
    event_number = 0
    warmup = 0
    round_prestart = 0
    begin_new_match = 0
    match_start = 0
    round_end = 0
    prestart_event = 0
    for event in events:
        event_number += 1
        if event['event_name'] =='round_announce_warmup':
            warmup = event_number
        if event['event_name'] =='round_prestart':
            round_prestart = event_number
        if event['event_name'] =='begin_new_match':
            begin_new_match = event_number
        if event['event_name'] =='round_announce_match_start':
            match_start = event_number
        if event['event_name'] =='round_end':
            round_end = event_number
        if warmup < round_prestart < begin_new_match < match_start < round_end:
            prestart_event = round_prestart
            break
    return prestart_event

client = MongoClient()
db = client.counter_strike
match_log = db.match_log

cwd = '/Users/johnwalters/ds/metis/projects/cs_go_stats'
os.chdir(cwd + "/demos_more")
demo_info_files = []
demo_event_files = []

for file in glob.glob("*_userinfo.txt"):
    demo_info_files.append(file)

for file in glob.glob("*_events.txt"):
    demo_event_files.append(file)
    
print len(demo_info_files)

match_name_number = [x[:-13] for x in demo_info_files]
match_name = [x[:-5] for x in match_name_number]


df_all = pd.DataFrame()
dict_all = []
relevant_events = ['player_death', 'weapon_fire', 'player_hurt', 'round_end', 'player_spawn', 'round_prestart', 'begin_new_match','round_announce_match_start', 'bomb_planted','bomb_defused','bomb_exploded']
for demo_file in tqdm(match_name_number):#tqdm(matches_test)
    match_name = demo_file[:-5]
    match_number = int(demo_file[-5:])
    #try:
    game_text = open(cwd +'/demos_more/' + demo_file + '_events.txt', 'r')
    game_info = open(cwd + '/demos_more/' + demo_file + '_userinfo.txt' , 'r').read().split("\n")
    map_name = find_map(game_info)
    steam_ids = player_info(game_info)
    game_data = game_text.read()
    game_data = game_data.split("\n")
    events = parent_child(game_data)
    start_event = find_match_begin(events)
    event_list = events_to_dict(steam_ids, events, match_name, match_number, map_name)
    dict_all = dict_all + event_list
    #except:
    #    failed_demos.append(demo_file)

#with open('all_demos_as_dict.pkl', 'w') as picklefile:
#            pickle.dump(dict_all, picklefile)

1112


In [2]:
df_all = pd.DataFrame.from_dict(dict_all)

In [17]:
len(df_all)

6636632

In [14]:
working_frame = df_all[(df_all.event_name == 'round_end')]

In [15]:
working_frame.keys()

Index([u'armor', u'assister', u'attacker', u'attacker_steamid', u'dmg_armor',
       u'dmg_health', u'dominated', u'event_name', u'event_number',
       u'headshot', u'health', u'hitgroup', u'map_name', u'match',
       u'match_number', u'message', u'penetrated', u'pitch', u'reason',
       u'revenge', u'round_number', u'silenced', u'site', u'steamid', u'team',
       u'teamnum', u'userid', u'weapon', u'weapon_fauxitemid',
       u'weapon_itemid', u'weapon_originalowner_xuid', u'winner', u'x', u'y',
       u'yaw', u'z'],
      dtype='object')

In [121]:
working_frame.groupby(['map_name', 'match', 'round_number', 'winner']).count().head()

armor  assister  \
map_name match                           round_number winner                    
de_cache 142_nerdrage-los-jefes_de_cache 1            2           0         0   
                                         2            3           0         0   
                                         3            3           0         0   
                                         4            3           0         0   
                                         5            3           0         0   

                                                              attacker  \
map_name match                           round_number winner             
de_cache 142_nerdrage-los-jefes_de_cache 1            2              0   
                                         2            3              0   
                                         3            3              0   
                                         4            3              0   
                                         5            3              0   

                                                              attacker_steamid  \
map_name match                           round_number winner                     
de_cache 142_nerdrage-los-jefes_de_cache 1            2                      0   
                                         2            3                      0   
                                         3            3                      0   
                                         4            3                      0   
                                         5            3                      0   

                                                              dmg_armor  \
map_name match                           round_number winner              
de_cache 142_nerdrage-los-jefes_de_cache 1            2               0   
                                         2            3               0   
                                         3            3               0   
                                         4            3               0   
                                         5            3               0   

                                                              dmg_health  \
map_name match                           round_number winner               
de_cache 142_nerdrage-los-jefes_de_cache 1            2                0   
                                         2            3                0   
                                         3            3                0   
                                         4            3                0   
                                         5            3                0   

                                                              dominated  \
map_name match                           round_number winner              
de_cache 142_nerdrage-los-jefes_de_cache 1            2               0   
                                         2            3               0   
                                         3            3               0   
                                         4            3               0   
                                         5            3               0   

                                                              event_name  \
map_name match                           round_number winner               
de_cache 142_nerdrage-los-jefes_de_cache 1            2                1   
                                         2            3                1   
                                         3            3                1   
                                         4            3                1   
                                         5            3                1   

                                                              event_number  \
map_name match                           round_number winner                 
de_cache 142_nerdrage-los-jefes_de_cache 1            2                  1   
                                         2      

In [21]:
df_all['event_name'].unique()

array(['player_spawn', 'weapon_fire', 'player_hurt', 'player_death',
       'round_prestart', 'begin_new_match', 'round_announce_match_start',
       'round_end', 'bomb_planted', 'bomb_exploded', 'bomb_defused'], dtype=object)

In [5]:
winner = df_all[(df_all.event_name == 'round_end')]
winner = winner[['map_name', 'match', 'round_number', 'winner']]
winner.head()

,map_name,match,round_number,winner
129,de_mirage,10_vega-squadron-rctic_de_mirage,1,3
461,de_mirage,10_vega-squadron-rctic_de_mirage,2,2
707,de_mirage,10_vega-squadron-rctic_de_mirage,3,2
993,de_mirage,10_vega-squadron-rctic_de_mirage,4,2
1229,de_mirage,10_vega-squadron-rctic_de_mirage,5,3


In [3]:
teams = df_all[(df_all.event_name == 'player_spawn')]
teams = teams[['match', 'map_name', 'round_number', 'event_number', 'userid', 'steamid', 'team']]
teams = teams.sort(['match', 'map_name', 'steamid', 'round_number'])
len(teams)

333375

In [4]:
new_round_number = 0
c_team = teams['team'].tolist()
c_id = teams['steamid'].tolist()
c_match = teams['match'].tolist()
c_map = teams['map_name'].tolist()
c_round = teams['round_number'].tolist()
c_user = teams['userid'].tolist()
p_team = c_team[0]
p_id = c_id[0]
p_match = c_match[0]
p_map = c_map[0]
team_event_number = []

real_match = []
real_match2 = []
y = 0
for event in tqdm(range(1, len(teams['event_number'].tolist()))):
    #c_round[event]
    #c_team[event]
    #c_id[event]
    #c_match[event]
    #c_map[event]
    y += 1
    p_team = c_team[event-1]
    p_id = c_id[event-1]
    p_match = c_match[event-1]
    p_map = c_map[event-1]
    p_round = c_round[event-1]
    if c_team[event] != p_team and c_id[event] == p_id and c_match[event] == p_match and c_map[event] == p_map and c_round[event] != p_round:
        team_event_number.append(event)
        x = 0
        for real_rounds in range(event-15, event+15):
            x += 1
            n_id = c_id[real_rounds]
            n_match = c_match[real_rounds]
            n_map = c_map[real_rounds]
            n_team = c_team[real_rounds]
            n_round = c_round[real_rounds]
            n_user = c_user[real_rounds]
            if n_id == c_id[event] and n_match == c_match[event] and n_map == c_map[event]:
                match_info = [n_id, n_match, n_map, n_team, n_user, n_round, x]
                real_match.append(match_info)
    #p_team = c_team[event]
    #p_id = c_id[event]
    #p_match = c_match[event]
    #p_map = c_map[event]

In [5]:
len(real_match)

285799

In [6]:
col = ['steamid', 'match', 'map_name', 'team', 'userid', 'round_number', 'real_rounds']
actual_df = pd.DataFrame(real_match, columns=col)

In [18]:
len(actual_df)

414

In [7]:
winner = df_all[(df_all.event_name == 'round_end')]
winner = winner[['map_name', 'match', 'round_number', 'winner']]
mapping = {'2': 'T', '3': 'CT'}
winner = winner.replace({'winner': mapping})

In [8]:
winner.head()

,map_name,match,round_number,winner
129,de_mirage,10_vega-squadron-rctic_de_mirage,1,CT
461,de_mirage,10_vega-squadron-rctic_de_mirage,2,T
707,de_mirage,10_vega-squadron-rctic_de_mirage,3,T
993,de_mirage,10_vega-squadron-rctic_de_mirage,4,T
1229,de_mirage,10_vega-squadron-rctic_de_mirage,5,CT


In [9]:
kills = df_all[(df_all.event_name == 'player_death')]
kills = kills[['attacker_steamid', 'attacker', 'map_name', 'match', 'round_number']]
r_kills = kills.groupby(['match', 'map_name', 'attacker_steamid', 'round_number']).count().reset_index()
r_kills.columns = ['match', 'map_name', 'steamid', 'round_number', 'kills']
deaths = df_all[(df_all.event_name == 'player_death')]
deaths = deaths[['steamid', 'userid', 'map_name', 'match', 'round_number']]
r_deaths = deaths.groupby(['match', 'map_name', 'steamid', 'round_number']).count().reset_index()
r_deaths.columns = ['match', 'map_name', 'steamid', 'round_number', 'deaths']
hits = df_all[(df_all.event_name == 'player_hurt')]
hits = hits[['attacker_steamid', 'attacker', 'map_name', 'match', 'round_number']]
r_hits = hits.groupby(['match', 'map_name', 'attacker_steamid', 'round_number']).count().reset_index()
r_hits.columns = ['match', 'map_name', 'steamid', 'round_number', 'hits']
shots = df_all[(df_all.event_name == 'weapon_fire')]
shots = shots[['steamid', 'userid', 'map_name', 'match', 'round_number']]
r_shots = shots.groupby(['match', 'map_name', 'steamid', 'round_number']).count().reset_index()
r_shots.columns = ['match', 'map_name', 'steamid', 'round_number', 'shots']
bomb_plant = df_all[(df_all.event_name == 'bomb_planted')]
bomb_plant = bomb_plant[['steamid', 'userid', 'map_name', 'match', 'round_number']]
r_bomb_plant = bomb_plant.groupby(['match', 'map_name', 'steamid', 'round_number']).count().reset_index()
r_bomb_plant.columns = ['match', 'map_name', 'steamid', 'round_number', 'bomb plant']
bomb_defused = df_all[(df_all.event_name == 'bomb_defused')]
bomb_defused = bomb_defused[['steamid', 'userid', 'map_name', 'match', 'round_number']]
r_bomb_defused = bomb_defused.groupby(['match', 'map_name', 'steamid', 'round_number']).count().reset_index()
r_bomb_defused.columns = ['match', 'map_name', 'steamid', 'round_number', 'bomb defused']

In [10]:
player_stats = actual_df.merge(r_kills, how='left', on=['match', 'map_name', 'round_number', 'steamid'])
player_stats = player_stats.sort(['match', 'map_name', 'steamid', 'real_rounds'])
player_stats = player_stats.merge(r_deaths, how='left', on=['match', 'map_name', 'round_number', 'steamid'])
player_stats = player_stats.sort(['match', 'map_name', 'steamid', 'real_rounds'])
player_stats = player_stats.merge(r_hits, how='left', on=['match', 'map_name', 'round_number', 'steamid'])
player_stats = player_stats.sort(['match', 'map_name', 'steamid', 'real_rounds'])
player_stats = player_stats.merge(r_shots, how='left', on=['match', 'map_name', 'round_number', 'steamid'])
player_stats = player_stats.sort(['match', 'map_name', 'steamid', 'real_rounds'])
player_stats = player_stats.merge(r_bomb_plant, how='left', on=['match', 'map_name', 'round_number', 'steamid'])
player_stats = player_stats.sort(['match', 'map_name', 'steamid', 'real_rounds'])
player_stats = player_stats.merge(r_bomb_defused, how='left', on=['match', 'map_name', 'round_number', 'steamid'])
player_stats = player_stats.sort(['match', 'map_name', 'steamid', 'real_rounds'])
#player_stats = player_stats.merge(winner, how='left', on=['match', 'map_name', 'round_number'])
#player_stats = player_stats.sort(['match', 'map_name', 'steamid', 'real_rounds'])

In [11]:
player_stats2 = player_stats.merge(winner, how='left', on=['match', 'map_name', 'round_number'])
player_stats2 = player_stats2.sort(['match', 'map_name', 'steamid', 'real_rounds'])

In [12]:
player_stats2

,steamid,match,map_name,team,userid,round_number,real_rounds,kills,deaths,hits,shots,bomb plant,bomb defused,winner
0,76561197960290296,10_vega-squadron-rctic_de_mirage,de_mirage,T,Twista * Logitech G (id:53),2,1,1,1,4,23,NaN,NaN,T
1,76561197960290296,10_vega-squadron-rctic_de_mirage,de_mirage,T,Twista * Logitech G (id:53),3,2,1,NaN,12,47,NaN,NaN,T
2,76561197960290296,10_vega-squadron-rctic_de_mirage,de_mirage,T,Twista * Logitech G (id:53),4,3,1,NaN,7,22,NaN,NaN,T
3,76561197960290296,10_vega-squadron-rctic_de_mirage,de_mirage,T,Twista * Logitech G (id:53),5,4,1,1,2,11,NaN,NaN,CT
4,76561197960290296,10_vega-squadron-rctic_de_mirage,de_mirage,T,Twista * Logitech G (id:53),6,5,1,1,4,19,NaN,NaN,T
5,76561197960290296,10_vega-squadron-rctic_de_mirage,de_mirage,T,Twista * Logitech G (id:53),7,6,3,NaN,8,28,NaN,NaN,T
6,76561197960290296,10_vega-squadron-rctic_de_mirage,de_mirage,T,Twista * Logitech G (id:53),8,7,NaN,NaN,2,24,NaN,NaN,T
7,76561197960290296,10_vega-squadron-rctic_de_mirage,de_mirage,T,Twista * Logitech G (id:53),9,8,1,NaN,2,13,NaN,NaN,T
8,76561197960290296,10_vega-squadron-rctic_de_mirage,de_mirage,T,Twista * Logitech G (id:53),10,9,NaN,NaN,3,14,NaN,NaN,T
9,76561197960290296,10_vega-squadron-rctic_de_mirage,de_mirage,T,Twista * Logitech G (id:53),11,10,3,NaN,16,26,1,NaN,T


In [53]:
player_stats2 = player_stats2.fillna(0)

In [14]:
with open('all_demos_as_df_best.pkl', 'w') as picklefile:
            pickle.dump(player_stats2, picklefile)

In [29]:
team_stats = player_stats2.groupby(['match', 'map_name', 'team', 'real_rounds', 'winner']).count().reset_index()
team_stats = team_stats[['match', 'map_name', 'team', 'real_rounds', 'winner', 'kills', 'deaths', 'hits', 'shots', 'bomb plant', 'bomb defused']]
team_stats.head()

,match,map_name,team,real_rounds,winner,kills,deaths,hits,shots,bomb plant,bomb defused
0,10_vega-squadron-rctic_de_mirage,de_mirage,CT,1,T,3,5,5,5,0,0
1,10_vega-squadron-rctic_de_mirage,de_mirage,CT,2,T,1,3,4,5,0,0
2,10_vega-squadron-rctic_de_mirage,de_mirage,CT,3,T,2,5,5,5,0,0
3,10_vega-squadron-rctic_de_mirage,de_mirage,CT,4,CT,2,3,4,5,0,0
4,10_vega-squadron-rctic_de_mirage,de_mirage,CT,5,T,3,5,4,5,0,0


In [31]:
t_team_stats = team_stats[(team_stats.team == 'T')]
t_team_stats = t_team_stats[['match', 'map_name', 'team', 'real_rounds', 'winner', 'kills', 'deaths', 'hits', 'shots', 'bomb plant']]
t_team_stats.columns = ['match', 'map_name', 't_team', 'real_rounds', 'winner', 't_kills', 't_deaths', 't_hits', 't_shots', 't_bomb_plant']
ct_team_stats = team_stats[(team_stats.team == 'CT')]
ct_team_stats = ct_team_stats[['match', 'map_name', 'team', 'real_rounds', 'winner', 'kills', 'deaths', 'hits', 'shots', 'bomb defused']]
ct_team_stats.columns = ['match', 'map_name', 'ct_team', 'real_rounds', 'winner', 'ct_kills', 'ct_deaths', 'ct_hits', 'ct_shots', 'ct_bomb_defuse']
team_stats_comp = t_team_stats.merge(ct_team_stats, how='left', on=['match', 'map_name', 'real_rounds', 'winner'])

In [34]:
len(team_stats_comp)

28563

In [52]:
player_stats2.head()

,steamid,match,map_name,team,userid,round_number,real_rounds,kills,deaths,hits,shots,bomb plant,bomb defused,winner
0,76561197960290296,10_vega-squadron-rctic_de_mirage,de_mirage,T,Twista * Logitech G (id:53),2,1,1,1,4,23,NaN,NaN,T
1,76561197960290296,10_vega-squadron-rctic_de_mirage,de_mirage,T,Twista * Logitech G (id:53),3,2,1,NaN,12,47,NaN,NaN,T
2,76561197960290296,10_vega-squadron-rctic_de_mirage,de_mirage,T,Twista * Logitech G (id:53),4,3,1,NaN,7,22,NaN,NaN,T
3,76561197960290296,10_vega-squadron-rctic_de_mirage,de_mirage,T,Twista * Logitech G (id:53),5,4,1,1,2,11,NaN,NaN,CT
4,76561197960290296,10_vega-squadron-rctic_de_mirage,de_mirage,T,Twista * Logitech G (id:53),6,5,1,1,4,19,NaN,NaN,T


In [ ]:
plot_kills = player_stats2[player_stats2.kills <= 5].kills

#plot_kills = plot_kills[plot_kills.kills <= 5]

plt.figure()

plt.xlabel('Kills')
plt.ylabel('Probability')
plot_kills.plot(kind='bar', alpha=.75)#, bins=[0, 1, 2, 3, 4, 5, 6]
savefig('kills.png')

In [ ]:
plot_deaths = player_stats2[player_stats2.deaths <= 1].deaths

#plot_kills = plot_kills[plot_kills.kills <= 5]

plt.figure()

plt.xlabel('Deaths')
plt.ylabel('Probability')
plot_deaths.plot(kind='bar', alpha=.75)#, bins=[0, 1, 2]
savefig('deaths.png')